use the data nba_logreg.csv . Each row represents an NBA player with their stats such as PTS (points per game),  BLK (blocks), STL (steals) etc. The last column shows whether the player's career length is more than 5 years (1) or not (0). Your task is to build a knn model that predicts whether a player's career length is more than 5 years or not.

In [92]:
# Run this cell first
from datascience import * 
%matplotlib inline
import matplotlib.pyplot as plots
import numpy as np
plots.style.use('fivethirtyeight')

 Load the data. We want to use the first 80% of the data for training and the last 20% for testing. Split the data.

In [93]:
# write your answer here
data = Table.read_table('nba_logreg.csv')
data

Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
Brandon Ingram,36,27.4,7.4,2.6,7.6,34.7,0.5,2.1,25,1.6,2.3,69.9,0.7,3.4,4.1,1.9,0.4,0.4,1.3,0
Andrew Harrison,35,26.9,7.2,2,6.7,29.6,0.7,2.8,23.5,2.6,3.4,76.5,0.5,2,2.4,3.7,1.1,0.5,1.6,0
JaKarr Sampson,74,15.3,5.2,2,4.7,42.2,0.4,1.7,24.4,0.9,1.3,67,0.5,1.7,2.2,1,0.5,0.3,1,0
Malik Sealy,58,11.6,5.7,2.3,5.5,42.6,0.1,0.5,22.6,0.9,1.3,68.9,1,0.9,1.9,0.8,0.6,0.1,1,1
Matt Geiger,48,11.5,4.5,1.6,3,52.4,0,0.1,0,1.3,1.9,67.4,1,1.5,2.5,0.3,0.3,0.4,0.8,1
Tony Bennett,75,11.4,3.7,1.5,3.5,42.3,0.3,1.1,32.5,0.4,0.5,73.2,0.2,0.7,0.8,1.8,0.4,0,0.7,0
Don MacLean,62,10.9,6.6,2.5,5.8,43.5,0,0.1,50,1.5,1.8,81.1,0.5,1.4,2,0.6,0.2,0.1,0.7,1
Tracy Murray,48,10.3,5.7,2.3,5.4,41.5,0.4,1.5,30,0.7,0.8,87.5,0.8,0.9,1.7,0.2,0.2,0.1,0.7,1
Duane Cooper,65,9.9,2.4,1,2.4,39.2,0.1,0.5,23.3,0.4,0.5,71.4,0.2,0.6,0.8,2.3,0.3,0,1.1,0
Dave Johnson,42,8.5,3.7,1.4,3.5,38.3,0.1,0.3,21.4,1,1.4,67.8,0.4,0.7,1.1,0.3,0.2,0,0.7,0


In [94]:
total_rows = data.num_rows
training_size = int(total_rows*0.8)
test_size = int(total_rows*0.2)
data_shuffled =data.shuffle()
training_size,test_size

(1072, 268)

In [95]:
train = data_shuffled.take(np.arange(0,training_size))
test = data_shuffled.take(np.arange(training_size,training_size+test_size))
train


Name,GP,MIN,PTS,FGM,FGA,FG%,3P Made,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,TARGET_5Yrs
Adreian Payne,32,23.1,6.7,2.8,6.9,41.4,0,0.3,11.1,0.9,1.4,65.2,1.5,3.6,5.1,0.9,0.6,0.3,1.4,0
Rasual Butler,72,21,7.5,2.9,7.9,36.2,0.7,2.4,29.2,1.1,1.4,73.1,0.4,2.2,2.6,1.3,0.3,0.6,1.1,1
Adrian Griffin,72,26.8,6.7,2.4,5.7,42.4,0.2,0.8,28.1,1.7,2.2,75.3,1.8,3.4,5.2,2.5,1.6,0.2,1.3,1
Bob Thornton,71,18.6,4.7,1.8,3.9,45.6,0,0,0,1.2,2.3,53.1,1.6,2.5,4.1,0.6,0.4,0.1,1.2,1
Howard Eisley,49,11.3,2.4,0.8,2.5,32.8,0.2,0.8,24.3,0.6,0.8,77.5,0.2,0.7,1,1.9,0.4,0.1,1,1
Tom Hammonds,61,13.2,5.3,2.1,4.8,43.7,0,0,0,1,1.6,64.3,1,1.8,2.8,0.8,0.2,0.2,0.8,1
Earl Boykins,22,10,3,1.4,3.6,38,0.1,0.8,16.7,0.1,0.1,66.7,0.3,0.5,0.8,1.5,0.3,0,0.9,1
Greg Monroe,80,27.8,9.4,3.8,6.9,55.1,0,0,0,1.8,2.9,62.2,3.1,4.4,7.5,1.3,1.2,0.6,1,1
Derrick Brown,57,9.4,3.3,1.2,2.6,46.3,0,0.1,28.6,0.8,1.3,66.7,0.5,0.8,1.4,0.3,0.4,0.2,0.3,0
Walt Szczerbiak,73,29.7,11.6,4.7,9.2,51.1,0.4,1.1,35.9,1.8,2.2,82.6,1.2,2.5,3.7,2.8,0.8,0.3,1.1,0


 Build a knn classifier which uses all numericals columns (except Target_5Yrs) as features to predict Target_5Yrs using the training data. Set the k parameter of knn to 3. Calculate your classifier's accuracy on your test data.

In [107]:
#write your answer here**


train_with_numeric=train.drop('Name')
 
def standard_units(x):
    x =np.array(x)
    return (x - np.mean(x))/np.std(x)

def standartUnitConverter(table):
    columnsize=table.num_columns
   
    table =Table()
   
    for i in range(0,columnsize):
       
        array=standard_units(table.column(0))
        table = table.with_column(str(i)+'.column',array)
        
        
    
    return table
    



def distance(data1,data2):
    
    data1=np.array(data1)
    data2=np.array(data2)

    
    return np.sqrt(np.sum((data1-data2)**2))
    
def distanceCalculator(train_with_numeric, new):
    data_without_target =train_with_numeric.drop('TARGET_5Yrs')
   
    def subDistance(row):
        return distance(np.array(new),np.array(row))
    return data_without_target.apply(subDistance)

def appending_sorted_DistancetoTable(data_without_target, new):
    
    return data_without_target.with_column('Distance',distanceCalculator(data_without_target, new)).sort('Distance')

def closest(data,newpoint,k):
    
    data_with_distance = appending_sorted_DistancetoTable(data,newpoint)
    taken_k=data_with_distance.take(np.arange(0,k))
    return taken_k

train_without_class=train_with_numeric.drop('TARGET_5Yrs')


# there are different approach to assign class for new point one of the is majority


def classDetection(train,new,k):


    taken = closest(train,new,k)
    def majority(taken_k):
        ones = taken_k.where('TARGET_5Yrs', are.equal_to(1)).num_rows
        zeros = taken_k.where('TARGET_5Yrs', are.equal_to(0)).num_rows
        if ones > zeros:
            return 1
        else:
            return 0

    return majority(taken)
    
classDetection(train_with_numeric,train_without_class.row(3),3)




1

In [108]:

test_with_numeric =test.drop('Name')


def accuracy(train_data,test_data,k):
    
    test_features = test_data.drop('TARGET_5Yrs')
    
    def detection(row):
        return classDetection(train_data,row,k)
    
    test_class_result = test_features.apply(detection)
    return test_data.with_column('Target 5 year Class',test_class_result)

table_with_predicted_class = accuracy(train_with_numeric,test_with_numeric,3)

def accuracy_level(table):
    number_row =table.num_rows
    
    target = table.column('TARGET_5Yrs')
    target_classication=table.column('Target 5 year Class')
    result = []
    for i in range(0,len(target)):
        if(target[i]==target_classication[i]):
            result.append(1)
        else:
            result.append(0)
    np_result= np.array(result)  
    count_true = np.count_nonzero(np_result==1)
    
    
    return count_true/number_row
    
accuracy_level(table_with_predicted_class)

    
    
    


0.6007462686567164

For the same model, try k = 1,2,3,4, and 5 and show which k value yields the best accuracy scores.

In [109]:
#write your answer here**

indek = np.arange(1,6)
result=[]

for i in indek:
    table_with_predicted_class = accuracy(train_with_numeric,test_with_numeric,i)
    result.append(accuracy_level(table_with_predicted_class))
    
result_table = Table().with_columns('K',indek,'Accuracy',result).sort('Accuracy')
result_table
    


K,Accuracy
2,0.529851
1,0.55597
3,0.600746
4,0.600746
5,0.649254


 We want to know which feautures are the most useful for this task. Set the value k to the value which gives the best performance in Question 3. Then for each feature you used previously, build a seperate knn model using the training data and all features except the corresponding feature, and calculate the performance on the test data.  

In [111]:


test_features= test_with_numeric.drop('TARGET_5Yrs')


dropped_column_indeks=np.arange(0,test_features.num_columns)
result=[]

for i in range(0,test_features.num_columns):
    table_with_predicted_class = accuracy(train_with_numeric.drop(i),test_with_numeric.drop(i),5)
    result.append(accuracy_level(table_with_predicted_class))
    
result_table = Table().with_columns('Dropped Column',dropped_column_indeks,'Accuracy After Drop',result)
result_table    
    
    
    
    
    
    
    




Dropped Column,Accuracy After Drop
0,0.61194
1,0.671642
2,0.664179
3,0.645522
4,0.652985
5,0.630597
6,0.649254
7,0.652985
8,0.660448
9,0.652985


In [113]:
#After drop some certain columns accuracy rate decrease, so these dropped column should be in the model

worst_result_table = result_table.where('Accuracy After Drop',are.below(0.649254)).sort('Accuracy After Drop',descending=False)

worst_result_table.column(0)




array([ 0, 11,  5, 13, 14,  3,  6, 10, 12, 16, 17, 18])

you have applied a leave-one-out method such that you exclude one feature and build the model with the rest. So if the performance of the model increases when we exclude a feature (compared to the model we use all features), that feature might not be a suitable feature for this task. So, exclude all features (if any) that yield higher performance when excluded than the model using all features. Then build the model with the remaining features. However, this time you will apply 5-fold cross validation. For cross-validation, do not shuffle the data. Report the accuracy for each fold.

In [117]:
data_dropped_columns = data.drop('Name').select( 0, 11,  5, 13, 14,  3,  6, 10, 12, 16, 17, 18,19)

def kfold(data,k1,k2):
    
    index = np.arange(data.num_rows)
    #shuffled_index = index.sample(with_replacement=False)
    test_size = int(data.num_rows/k1)
    total_accuracy = 0
    
    for i in np.arange(k1):
        
        test_index = index[i*test_size:(i+1)*test_size]
        train_index = np.setdiff1d(index, test_index)
        test = data.take(test_index)
        train = data.take(train_index)
        
        
        table_with_predicted_class = accuracy(train,test,5) 
        dogruluk_orani=accuracy_level(table_with_predicted_class)
        total_accuracy=dogruluk_orani+total_accuracy
        print(str(i+1)+'th Fold result'+str(dogruluk_orani))
    return total_accuracy /k1

print('K fold result '+str(kfold(data_dropped_columns,5,5))) 
        
    
    
    
    
    
    



1th Fold result0.5932835820895522
2th Fold result0.6791044776119403
3th Fold result0.6567164179104478
4th Fold result0.6567164179104478
5th Fold result0.6567164179104478
K fold result 0.6485074626865672
